In [13]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

url = "https://api.seatgeek.com/2/venues"
client_id = os.getenv("client_id_venues")

def get_kc_metro_venues(client_id, url):
    results = []
    per_page = 100
    states = [ "MO", "KS"]

    for city, state in states:
        page = 1
        while True:
            params = {
                "city": "Kansas City",
                "state": states,
                "client_id": client_id,
                "page": page,
                "per_page": per_page
            }

            res = requests.get(url, params=params)

            if res.status_code != 200:
                print(f"Error fetching {city}, {state}: {res.status_code} - {res.text}")
                break

            data = res.json()
            venues = data.get("venues", [])
            results.extend(venues)

            print(f"Fetched page {page} for {city}, {state} with {len(venues)} venues")

            if len(venues) < per_page:
                break

            page += 1

    return results

kc_venues = get_kc_metro_venues(client_id, url)

Fetched page 1 for M, O with 100 venues
Fetched page 2 for M, O with 74 venues
Fetched page 1 for K, S with 100 venues
Fetched page 2 for K, S with 74 venues


In [14]:
import pandas as pd
#JSON is then Normalized into flattened data file 
KC_venues_df = pd.json_normalize(kc_venues)

KC_venues_df.head()

,id,name,name_v2,postal_code,timezone,score,popularity,address,country,city,...,extended_address,display_location,access_method,has_upcoming_events,num_upcoming_events,url,relative_url,location.lat,location.lon,stats.event_count
0,4112,Children's Mercy Park,Children's Mercy Park,66111,America/Chicago,0.79,38203,1 Sporting Way,US,Kansas City,...,"Kansas City, KS 66111","Kansas City, KS",None,True,5,https://seatgeek.com/venues/children-s-mercy-p...,/venues/children-s-mercy-park/tickets,39.1216,-94.8232,5
1,1643,Kansas Speedway,Kansas Speedway,66111,America/Chicago,0.62,1453,400 Speedway Blvd.,US,Kansas City,...,"Kansas City, KS 66111","Kansas City, KS",None,True,3,https://seatgeek.com/venues/kansas-speedway/ti...,/venues/kansas-speedway/tickets,39.1151,-94.8331,3
2,2064,Hartman Arena,Hartman Arena,67147,America/Chicago,0.53,159,8151 N. Hartman Arena Drive,US,Park City,...,"Park City, KS 67147","Park City, KS",None,True,2,https://seatgeek.com/venues/hartman-arena/tickets,/venues/hartman-arena/tickets,37.8267,-97.3310,2
3,4138,United Wireless Arena,United Wireless Arena,67801,America/Chicago,0.52,136,4100 W Comanche,US,Dodge City,...,"Dodge City, KS 67801","Dodge City, KS",None,True,4,https://seatgeek.com/venues/united-wireless-ar...,/venues/united-wireless-arena/tickets,37.7666,-100.0620,4
4,3012,Legends Field,Legends Field,66111,America/Chicago,0.49,1047,1800 Village West Parkway,US,Kansas City,...,"Kansas City, KS 66111","Kansas City, KS",None,True,16,https://seatgeek.com/venues/legends-field/tickets,/venues/legends-field/tickets,39.1240,-94.8320,16


In [ ]:
KC_venues_df.info()
KC_venues_df = KC_venues_df[KC_venues_df['has_upcoming_events'] == True]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   348 non-null    int64  
 1   name                 348 non-null    object 
 2   name_v2              348 non-null    object 
 3   postal_code          348 non-null    object 
 4   timezone             348 non-null    object 
 5   score                348 non-null    float64
 6   popularity           348 non-null    int64  
 7   address              180 non-null    object 
 8   country              348 non-null    object 
 9   city                 348 non-null    object 
 10  state                348 non-null    object 
 11  slug                 348 non-null    object 
 12  metro_code           348 non-null    int64  
 13  capacity             348 non-null    int64  
 14  links                348 non-null    object 
 15  passes               348 non-null    obj

In [16]:
#Venue is then saved to a file folder so it can be called for the event's scripts. 
#this file wont be updated frequently, due to the nature of venues not changing frequently
KC_venues_df.to_csv("events_data/venues_data/KC_Venues.csv",index=False)